# Extending Variational Boosting to Multiple Competing Variational Families

This jupyter notebook contains all of the code used to generate the plots in the report.

In [1]:
using Plots, LinearAlgebra, ReverseDiff, ForwardDiff, SpecialFunctions, Distributions, Statistics, Random

## ADVI

We begin by implementing the ADVI algorithm that can be used to obtain the first component in the mixture approximation.

In [1305]:
function ADVI(logπ, grad, γ, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig)
    μs = zeros(d, num_iter)
    Σs = zeros(d, d, num_iter)
    obj = zeros(num_iter)

    for k=1:num_iter
        # transform ψ to mean and covariance
        μ, L = transform(ψ0, diag_sig, d)
        # store parameter
        μs[:,k] = μ
        Σs[:,:,k] = L * transpose(L)
        # sample from κ
        y = rand(MvNormal(d, 1), Ngrad)
        # compute the gradient
        ∇ψ = grad(y, ψ0, logπ, Ngrad, diag_sig, d)
        # take a step
        ψ0 = ψ0 .- γ(k) .* ∇ψ
        # estimate the objective function
        μ, L = transform(ψ0, diag_sig, d)
        xobj = rand(MvNormal(μ, L * transpose(L)), Nobj)
        for i=1:size(xobj)[1]
            obj[k] += logpdf(MvNormal(μ, L * transpose(L)), xobj[:,i]) - logπ(xobj[:,i])
        end
        obj[k] /= Nobj
    end

    return μs, Σs, obj
end

ADVI (generic function with 3 methods)

In [1306]:
function transform(ψ, diag_sig, d)
    μ = ψ[1:d]
    if diag_sig
        L = Diagonal(exp.(ψ[d+1:d+d]./2))
    else
        L = reshape(ψ[d+1:end], (d, d))
    end
    return μ, L
end

function logπψ(y, ψ0, logπ, diag_sig, d)
    μ, L = transform(ψ0, diag_sig, d)
    x = vec(μ .+ L * y)
    return -(logπ(x) .+ log(abs(det(L))))
end

function grad(ys, ψ0, logπ, Ngrad, diag_sig, d)
    ret = zeros(size(ψ0)[1])
    for i in 1:Ngrad
        y = vec(ys[:,i])    
        ret = ret .+ ForwardDiff.gradient(ψψ -> logπψ(y, ψψ, logπ, diag_sig, d), ψ0)
    end
    return (ret./Ngrad)
end

grad (generic function with 1 method)

## Variational Boosting

We now implement the variational boosting algorithm. First we implement the weighted EM method for initializing new components.

In [1307]:
# weighted EM helper
function sample_from_current_mixture(μs, Σs, λs, Ngrad, d)
    xs = zeros(d, Ngrad)
    num_comp = size(μs)[2]
    for i in 1:Ngrad
        # identify component
        u = rand(Uniform(0,1))
        comp = 0
        for j in 1:num_comp
            if u <= sum(λs[1:j])
                comp = j
                break
            end
        end
        # sample from component
        xs[:,i] = rand(MvNormal(vec(μs[:,comp]), Σs[:,:,comp]))
    end
    return xs
end

sample_from_current_mixture (generic function with 2 methods)

In [1308]:
# weighted EM helper
function log_mixture(μs, Σs, λs, x)
    mix = 0.
    num_comp = size(μs)[2]
    for i in 1:num_comp
        mix = mix + λs[i] * pdf(MvNormal(vec(μs[:,i]), Σs[:,:,i]), vec(x))
    end
    return log(mix)
end

log_mixture (generic function with 2 methods)

In [1309]:
# weighted EM helper
function compute_EM_weights(μs, Σs, λs, logπ, xs)
    N = size(xs)[2]
    logws = zeros(N)
    for i in 1:N
        x = xs[:,i]
        logws[i] = logπ(x) - log_mixture(μs, Σs, λs, x)
    end
    
    println(exp.(logws))
    return exp.(logws)
end

compute_EM_weights (generic function with 1 method)

In [1310]:
# weighted EM helper
function weighted_EM_initialization(μs, Σs, λs, weights, xs, diag_sig)
    num_comp = size(μs)[2] + 1
    N = size(xs)[2]
    d = size(μs)[1]
    λ = 1. / num_comp
    μ = vec(zeros(d))
    Σ = Diagonal(ones(d))
    # EM loop
    keep = true
    while(keep)
        # E step
        resp = zeros(N, num_comp)
        λs_curr = (1 - λ) .* λs
        for i in 1:N
            x = vec(xs[:,i])
            for j in 1:num_comp
                if j < num_comp
                    resp[i,j] = pdf(MvNormal(vec(μs[:,j]), Σs[:,:,j]), x) * λs_curr[j]
                else
                    resp[i,j] = pdf(MvNormal(vec(μ),Σ), x) * λ
                end
            end
        end
        # normalize by dividing row sum
        rowsum = sum(resp, dims=2)
        resp = resp ./ rowsum
        # M step
        Nk = sum(resp, dims=1)[num_comp]
        newλ = Nk / N
        newμ = vec(zeros(d))
        newΣ = zeros(d,d)
        # update μ
        for i in 1:N
            newμ = newμ .+ (1. / Nk) .* resp[i, num_comp] .* weights[i] .* vec(xs[:,i])
        end
        # update Σ
        for i in 1:N
            diff = vec(xs[:,i]) .- newμ
            newΣ = newΣ .+ (1. / Nk) .* resp[i, num_comp] .* weights[i] .* (diff) * transpose(diff)
        end
        # ensure Σ is Hermitian
        for j in 1:d
            for k in 1:j
                newΣ[j,k] = newΣ[k,j]
            end
        end
        # check convergence
        if (newλ - λ)^2 <= 1e-3
            println("finished initializing ψ and λ")
            keep = false
        end
        # update params
        λ = newλ
        μ = newμ
        Σ = newΣ
    end
    # modify Σ to follow specification of diag_sig
    if diag_sig
        Σ = Diagonal(Σ)
    end

    return λ, μ, Σ
end

weighted_EM_initialization (generic function with 3 methods)

The following functions compute the gradient for component optimization.

In [1311]:
# component optimization gradient helper
function log_mix(x, μs, Σs, λs, μ_opt, L_opt, λ)
    ret = 0.
    for i in 1:size(μs)[2]
        ret = ret + (1 - λ) * λs[i] * pdf(MvNormal(vec(μs[:,i]), Σs[:,:,i]), vec(x))
    end
    ret = ret + λ * pdf(MvNormal(vec(μ_opt), L_opt * transpose(L_opt)), vec(x))
    return log(ret)
end

log_mix (generic function with 2 methods)

In [1312]:
# component optimization gradient helper
function obj_comp(y, ψ, λ, logπ, diag_sig, d, μs, Σs, λs)
    μ_opt, L_opt = transform(ψ, diag_sig, d)
    num_comp = size(μs)[2] + 1
    ret = 0.
    for i in 1:num_comp
        if i < num_comp
            if diag_sig
                L = Diagonal(sqrt.(Σs[:,:,i]))
            else
                C = cholesky(Σs[:,:,i])
                L = convert(Array{Float64,2}, C.L)
            end
            x = vec(μs[:,i] .+ L * y)
            # ln (mixture approx) - ln (π)
            ret = ret - (1-λ) * λs[i] * logπ(x)
            ret = ret + (1-λ) * λs[i] * log_mix(x, μs, Σs, λs, μ_opt, L_opt, λ)
        else
            x = vec(μ_opt .+ L_opt * y)
            # ln (mixture approx) - ln (π)
            ret = ret - λ * logπ(x)
            ret = ret + λ * log_mix(x, μs, Σs, λs, μ_opt, L_opt, λ)
        end
    end
    
    return ret
end

obj_comp (generic function with 2 methods)

In [1313]:
# component optimization gradient helper
function grad_comp(ys, ψ, λ, logπ, Ngrad, diag_sig, d, μs, Σs, λs)
    retψ = zeros(size(ψ)[1])
    retλ = 0.
    for i in 1:Ngrad
        y = vec(ys[:,i])
        retψ = retψ .+ ForwardDiff.gradient(ψψ -> obj_comp(y, ψψ, λ, logπ, diag_sig, d, μs, Σs, λs), ψ)
        retλ = retλ + ForwardDiff.derivative(λλ -> obj_comp(y, ψ, λλ, logπ, diag_sig, d, μs, Σs, λs), λ)
    end
    return (retψ./Ngrad), (retλ/Ngrad)
end

grad_comp (generic function with 2 methods)

The following function performs the component optimization.

In [1314]:
function optimize_component(μs, Σs, λs, λ_k, μ_k, Σ_k, logπ, γ, Ngrad, d, num_iter, transform, diag_sig)
    # turn μ_k and Σ_k to the format of ψ
    ψ0 = copy(μ_k)
    if diag_sig
        append!(ψ0, log.([Σ_k[i,i] for i in 1:d]))
    else
        C = cholesky(Σ_k)
        append!(ψ0, vec(C.L))
    end
    
    ψ = vec(copy(ψ0))
    println(ψ)
    println(typeof(ψ))

    # start optimization
    λks = zeros(2*num_iter)
    μks = zeros(d, num_iter)
    Σks = zeros(d, d, num_iter)
    λ = λ_k
    
    for k=1:(2*num_iter)
        if k <= num_iter
            # transform ψ to mean and covariance
            μ, L = transform(ψ, diag_sig, d)
            # store parameter
            μks[:,k] = μ
            Σks[:,:,k] = L * transpose(L)
        end

        λks[k] = λ
        
        # sample from κ
        y = rand(MvNormal(d, 1), Ngrad)
        
        # compute the gradient
        ∇ψ, ∇λ = grad_comp(y, ψ, λ, logπ, Ngrad, diag_sig, d, μs, Σs, λs)
        # take a step
        if k <= num_iter
            ψ = ψ .- γ(k) .* ∇ψ
        end
        λ = λ - (γ(k)/200) * ∇λ
        # project λ to feasible region
        if λ < 0.
            λ = 0.
        elseif λ > 1.
            λ = 1.
        end
    end

    println("-----")
    y = rand(MvNormal(d, 1), Ngrad)
    ∇ψ, ∇λ = grad_comp(y, ψ, λ, logπ, Ngrad, diag_sig, d, μs, Σs, λs)
    println("ψ gradient: ", ∇ψ)
    println("λ gradient: ", ∇λ)
    println("-----")
    
    return λks[2*num_iter], μks[:,num_iter], Σks[:,:,num_iter]
end

optimize_component (generic function with 2 methods)

The following function performs variational boosting, where the first component is optimized using ADVI.

In [1315]:
function VB(logπ, grad, γ, Ncomp, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig)
    μs = zeros(d, Ncomp)
    Σs = zeros(d, d, Ncomp)
    λs = [1.]
    objs = zeros(Ncomp)

    # fitting first component
    println("optimizing component 1 / ", Ncomp)
    mus, sigmas, obj = ADVI(logπ, grad, γ, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig)
    μs[:,1] = mus[:,num_iter]
    Σs[:,:,1] = sigmas[:,:,num_iter]
    println("component 1")
    println("mean: ", μs[:,1])
    println("variance: ", Σs[:,:,1])
    println("-----")

    # fitting remaining components
    for k in 2:Ncomp
        println("optimizing component ", k, " / ", Ncomp)
        # init new component
        xs = sample_from_current_mixture(μs[:,1:k-1], Σs[:,:,1:k-1], λs[1:k-1], 40, d)
        weights = compute_EM_weights(μs[:,1:k-1], Σs[:,:,1:k-1], λs[1:k-1], logπ, xs)
        λ_k, μ_k, Σ_k = weighted_EM_initialization(μs[:,1:k-1], Σs[:,:,1:k-1], λs[1:k-1], weights, xs, diag_sig)
        println("initialization of weight, mean, and variance")
        println(λ_k)
        println(μ_k)
        println(Σ_k)
        println("-----")

        # optimize new component
        λ_k, μ_k, Σ_k = optimize_component(μs[:,1:k-1], Σs[:,:,1:k-1], λs[1:k-1], λ_k, μ_k, Σ_k,
                                           logπ, γ, Ngrad, d, num_iter, transform, diag_sig)
        # update parameters
        λs = (1. - λ_k) .* λs
        push!(λs, λ_k)
        μs[:,k] .= μ_k
        Σs[:,:,k] .= Σ_k

        println("component ", k)
        println("mean: ", μs[:,k])
        println("variance: ", Σs[:,:,k])
        println("weight: ", λ_k)
        println("-----")
    end

    return μs, Σs, λs
end

VB (generic function with 3 methods)

We first demonstrate a case where the weighted EM algorithm fails.

In [1316]:
μ = vec([2. 2.])
Σ = [1 0.; 0. 1]
logπ = x -> log(0.5 * pdf(MvNormal(μ, Σ), x) + 0.5 * pdf(MvNormal(-μ, Σ), x))
γ = k -> 0.1 /sqrt(k)
Ngrad = 200
Nobj = 200
ψ0 = vec([1.5 1.5 1. 1.])
diag_sig = true
d = 2
num_iter = 1000
Ncomp = 2

Random.seed!(1)
μs, Σs, λs = VB(logπ, grad, γ, Ncomp, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig);

optimizing component 1 / 2
component 1
mean: [1.9707038202545093, 1.9694340881629466]
variance: [1.0992736232511053 0.0; 0.0 1.1026546848108385]
-----
optimizing component 2 / 2
[0.5327189451215595, 0.5017136131829105, 0.540461719305902, 0.48381084318745093, 0.5521877769390933, 0.5207485482723334, 0.5486813198544986, 0.5062282961260438, 0.41576286968420956, 0.49790262773233823, 0.47172198807627275, 0.5375937775272542, 0.4951387501887072, 0.5178332480708243, 0.5223591431477888, 0.5186392865899537, 0.5510055950817563, 0.49205109423039295, 0.4940926564614897, 0.47845911540806024, 0.5236116175830913, 0.4346887122483684, 0.4993523407260072, 0.4837488994332491, 0.5479484389430723, 0.4399662455292501, 0.5357114546778512, 0.5322096263902948, 0.5155967618334488, 0.4442263986538946, 0.394379519583017, 0.504200687695202, 0.5551649393906588, 0.5208367133116818, 0.5532140135104433, 0.5291721601183221, 0.486076318386388, 0.44590770269986657, 0.51257389028276, 0.4779403373593588]
finished initializin

InterruptException: [91mInterruptException:[39m

In [1317]:
for i in 1:Ncomp
    println("mu: ", μs[:,i])
    println("sigma: ", Σs[:,:,i])
    println("lambda: ", λs[i])
end

mu: [1.9707038202545093, 1.9694340881629466]
sigma: [1.0992736232511053 0.0; 0.0 1.1026546848108385]
lambda: 0.9983550729078866
mu: [0.36434653135468625, 0.37154584859953504]
sigma: [0.16489604097628668 0.0; 0.0 0.13619115843002563]
lambda: 0.0016449270921134356


In [1318]:
function πtrue(x,y)
    return 0.5 * pdf(MvNormal(μ, Σ), vec([x y])) + 0.5 * pdf(MvNormal(-μ, Σ), vec([x y]))
end

function πapprox(x,y)
    ret = 0.
    for i in 1:Ncomp
        ret = ret + λs[i] * pdf(MvNormal(μs[:,i], Σs[:,:,i]), vec([x y]))
    end
    return ret
end

x1s = -5:0.1:5
x2s = -5:0.1:5

contour(x1s, x2s, (x, y) -> πtrue(x,y), xlabel="x1", ylabel="x2", lw=1, levels=30)
png("true_post_far")

In [1319]:
contour(x1s, x2s, (x, y) -> πapprox(x,y), lw=1, levels=30, xlabel="x1", ylabel="x2")
png("approx_post_far")

We hence discard the weighted EM initialization and instead intialize the next component with a component centred at the origin with a large variance.

In [1320]:
function VB(logπ, grad, γ, Ncomp, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig)
    μs = zeros(d, Ncomp)
    Σs = zeros(d, d, Ncomp)
    λs = [1.]
    objs = zeros(Ncomp)

    # fitting first component
    println("optimizing component 1 / ", Ncomp)
    mus, sigmas, obj = ADVI(logπ, grad, γ, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig)
    μs[:,1] = mus[:,num_iter]
    Σs[:,:,1] = sigmas[:,:,num_iter]
    println("component 1")
    println("mean: ", μs[:,1])
    println("variance: ", Σs[:,:,1])
    println("-----")
    
    # fitting remaining components
    for k in 2:Ncomp
        println("optimizing component ", k, " / ", Ncomp)
        # init new component
        λ_k = 1. / Ncomp
        Σ_k = Diagonal(ones(d))
        idx = Int64(floor(rand()*k))+1
        μ_k = -vec(ones(d))

        # optimize new component
        λ_k, μ_k, Σ_k = optimize_component(μs[:,1:k-1], Σs[:,:,1:k-1], λs[1:k-1], λ_k, μ_k, Σ_k,
                                           logπ, γ, Ngrad, d, num_iter, transform, diag_sig)
        # update parameters
        λs = (1. - λ_k) .* λs
        push!(λs, λ_k)
        μs[:,k] .= μ_k
        Σs[:,:,k] .= Σ_k

        println("component ", k)
        println("mean: ", μs[:,k])
        println("variance: ", Σs[:,:,k])
        println("weight: ", λ_k)
        println("-----")
    end

    return μs, Σs, λs
end

VB (generic function with 3 methods)

In [1321]:
μ = vec([3. 3.])
Σ = [1 0.1; 0.1 1]
logπ = x -> log(0.5 * pdf(MvNormal(μ, Σ), x) + 0.5 * pdf(MvNormal(-μ, Σ), x))
γ = k -> 0.1 /sqrt(k)
Ngrad = 200
Nobj = 200
ψ0 = vec([1.5 1.5 1. 1.])
diag_sig = true
d = 2
num_iter = 1000
Ncomp = 2

Random.seed!(1)
μs, Σs, λs = VB(logπ, grad, γ, Ncomp, Ngrad, Nobj, ψ0, d, num_iter, transform, diag_sig);

optimizing component 1 / 2
component 1
mean: [2.9941526171455397, 2.9928674816328695]
variance: [1.0257877522063494 0.0; 0.0 1.0299760983657085]
-----
optimizing component 2 / 2
[-1.0, -1.0, 0.0, 0.0]
Array{Float64,1}
-----
ψ gradient: [0.06557776016618942, 0.05759889421271219, 0.06281868025584292, 0.002957233010158445]
λ gradient: -0.059942449696490326
-----
component 2
mean: [-2.85627062883778, -2.8546278764693818]
variance: [1.016294174541848 0.0; 0.0 1.0195371494456904]
weight: 0.4808757132944286
-----


In [1322]:
for i in 1:Ncomp
    println("mu: ", μs[:,i])
    println("sigma: ", Σs[:,:,i])
    println("lambda: ", λs[i])
end

mu: [2.9941526171455397, 2.9928674816328695]
sigma: [1.0257877522063494 0.0; 0.0 1.0299760983657085]
lambda: 0.5191242867055714
mu: [-2.85627062883778, -2.8546278764693818]
sigma: [1.016294174541848 0.0; 0.0 1.0195371494456904]
lambda: 0.4808757132944286


In [1323]:
function πtrue(x,y)
    return 0.5 * pdf(MvNormal(μ, Σ), vec([x y])) + 0.5 * pdf(MvNormal(-μ, Σ), vec([x y]))
end

function πapprox(x,y)
    ret = 0.
    for i in 1:Ncomp
        ret = ret + λs[i] * pdf(MvNormal(μs[:,i], Σs[:,:,i]), vec([x y]))
    end
    return ret
end

x1s = -6:0.1:6
x2s = -6:0.1:6

contour(x1s, x2s, (x, y) -> πtrue(x,y), xlabel="x1", ylabel="x2", lw=1, levels=30)
png("true_post_close")

In [1324]:
contour(x1s, x2s, (x, y) -> πapprox(x,y), lw=1, levels=30, xlabel="x1", ylabel="x2")
png("approx_post_close")

## Extending to Multiple Variational Families

For this extension we focus on approximating mixture of Beta posteriors using log-normal and exponential components. We first implement the transformations so that the support of the variational family matches that of the target.

In [3]:
# takes t in (0,∞) and constrains it to be in (0,1)
function constrain(t)
    x = 1/(t+1)
    logJ = -2 * log(t+1)
    return x, logJ
end

function unconstrain(x)
    t = (1. / x) - 1
    logJ = -2 * log(x)
    return t, logJ
end

unconstrain (generic function with 1 method)

We now generalize the implementation of ADVI to both families of distributions.

In [236]:
function ADVI(logπ, grad, γ, Ngrad, ψ0, num_iter, constrain, κ, family)
    # build the log density ρ that has unconstrained support
    transform_logπ = (x, logJ) -> logπ(x) + logJ
    logρ = x -> transform_logπ(constrain(x)...)
    # store parameter values
    if family == 1
        ψs = zeros(2, num_iter)
    elseif family == 2
        ψs = zeros(num_iter)
    end
    #main loop
    for k=1:num_iter
        # store parameters
        if family == 1
            ψs[:,k] = ψ0
        elseif family == 2
            ψs[k] = ψ0
        end
        # sample from κ
        ys = κ(Ngrad)
        # compute the gradient
        ∇ψ = grad(ys, ψ0, logρ, Ngrad)
        # take a step
        ψ0 = @. ψ0 - γ(k) * ∇ψ
        if family == 1 && ψ0[2] < 0
            ψ0[2] = 0.
        elseif family == 2 && ψ0 < 0
            ψ0 = 0.
        end
    end
    return ψs
end

ADVI (generic function with 1 method)

We now implement the functions that are specific to each variational family.

In [237]:
function κ(Nobj)
    return rand(Uniform(0,1), Nobj)
end

## log-normal

function logρψ_gaussian(y, ψ, logρ)
    x = exp(ψ[1] + sqrt(2 * ψ[2]^2) * erfinv(2 * y - 1))
    trans = yy -> exp(ψ[1] + sqrt(2 * ψ[2]^2) * erfinv(2 * yy - 1))
    g = ForwardDiff.derivative(yy -> trans(yy), y)
    
    return -(logρ(x) + log(abs(g)))
end

function grad_gaussian(ys, ψ0, logρ, Ngrad)
    ret = zeros(2)
    for i in 1:Ngrad
        y = ys[i]
        ret = ret .+ ForwardDiff.gradient(ψψ -> logρψ_gaussian(y, ψψ, logρ), ψ0)
    end
    return (ret ./ Ngrad)
end

function sample_gaussian_q(N, ψ)
    y = rand(Uniform(), N)
    t = @. exp(ψ[1] + sqrt(2 * ψ[2]^2) * erfinv(2 * y - 1))
    samps = zero(t)
    for i=1:size(samps)[1]
        samps[i] = constrain(t[i])[1]
    end
    return samps
end

function logq_gaussian(x, ψ)
    t, logJ = unconstrain(x) 
    log_f = logpdf(LogNormal(ψ[1], ψ[2]), t)
    return logJ + log_f
end

## exponential

function logρψ_exponential(y, ψ, logρ)
    x = -log(1 - y) / ψ
    return -(logρ(x) + log(abs(ψ^-1 + 1/(1-y))))
end

function grad_exponential(ys, ψ0, logρ, Ngrad)
    ret = 0.
    for i in 1:Ngrad
        y = ys[i]
        ret = ret + ForwardDiff.derivative(ψψ -> logρψ_exponential(y, ψψ, logρ), ψ0)
    end
    return (ret / Ngrad)
end

function sample_exponential_q(N, ψ)
    y = rand(Uniform(), N)
    t = @. -log(1 - y) / ψ
    samps = zero(t)
    for i=1:size(samps)[1]
        samps[i] = constrain(t[i])[1]
    end
    return samps
end

function logq_exponential(x, ψ)
    t, logJ = unconstrain(x) 
    log_f = logpdf(Exponential(ψ), t)
    return logJ + log_f
end

logq_exponential (generic function with 1 method)

We check the performance of ADVI. Note even with the exponential distribution, the approximation's density decreases as the input gets close to $0$.

In [279]:
logπ = x -> logpdf(Beta(1,5), x)
γ = k -> 0.1/sqrt(k)
Nobj = 20
ψ0 = 0.5
num_iter = 1000
family = 2

ψs = ADVI(logπ, grad_exponential, γ, Ngrad, ψ0, num_iter, constrain, κ, family);

println(ψs[num_iter])

samples = sample_exponential_q(5000, ψs[num_iter])
histogram(samples, normed=true,bins=50, alpha=0.3, lavel = "approximation")
plot!(0:0.001:1, x -> pdf(Beta(1, 5), x), label = "target", linewidth=1.5)
png("exp_single")

0.10970529530300867


In [280]:
logπ = x -> logpdf(Beta(1,5), x)
γ = k -> 0.1/sqrt(k)
Nobj = 20
ψ0 = vec([0. 1.])
num_iter = 1000
family = 1

ψs = ADVI(logπ, grad_gaussian, γ, Ngrad, ψ0, num_iter, constrain, κ, family);

println(ψs[:,num_iter])

samples = sample_gaussian_q(5000, ψs[:,num_iter])
# samples = sample_gaussian_q(5000, [0. 0.5])
histogram(samples, normed=true,bins=50, alpha=0.3, label = "approximation")
plot!(0:0.001:1, x -> pdf(Beta(1, 5), x), label = "target", linewidth=1.5)
png("gauss_single")

[1.9731540152499398, 1.1769161604675944]


We now implement Variational Boosting that considers both Gaussian and exponential components

In [135]:
# component optimization gradient helper
# families: 1 encodes gaussian and 2 encodes exponential
function log_mix(x, ψs, λs, families, ψ_opt, λ, family_opt)
#     println(ψs)
    ret = 0.
    for i in 1:size(families)[1]
        if families[i] == 1
#             println(ψs[i][2])
            ret = ret + (1 - λ) * λs[i] * pdf(LogNormal(ψs[i][1], ψs[i][2]), x)
        else
            ret = ret + (1 - λ) * λs[i] * pdf(Exponential(ψs[i]), x)
        end
    end
    
    if family_opt == 1
#         println(ψ_opt[2])
        ret = ret + λ * pdf(LogNormal(ψ_opt[1], ψ_opt[2]), x)
    else
        ret = ret + λ * pdf(Exponential(ψ_opt), x)
    end
    
    return log(ret)
end

log_mix (generic function with 1 method)

In [136]:
# component optimization gradient helper
function obj_comp(y, ψ_opt, λ, logρ, ψs, λs, families, family_opt)
    num_comp = size(families)[1] + 1
    ret = 0.
    for i in 1:num_comp
        if i < num_comp
            if families[i] == 1
                x = exp(ψs[i][1] + sqrt(2 * ψs[i][2]^2) * erfinv(2 * y - 1))
            else
                x = -log(1 - y) / ψs[i]
            end
            # ln (mixture approx) - ln (π)
            ret = ret - (1-λ) * λs[i] * logρ(x)
            ret = ret + (1-λ) * λs[i] * log_mix(x, ψs, λs, families, ψ_opt, λ, family_opt)
        else
            if family_opt == 1
                x = exp(ψ_opt[1] + sqrt(2 * ψ_opt[2]^2) * erfinv(2 * y - 1))
            else
                x = -log(1 - y) / ψ_opt
            end
            # ln (mixture approx) - ln (π)
            ret = ret - λ * logρ(x)
            ret = ret + λ * log_mix(x, ψs, λs, families, ψ_opt, λ, family_opt)
        end
    end
    
    return ret
end

obj_comp (generic function with 1 method)

In [177]:
# component optimization gradient helper
function grad_comp(ys, ψ, λ, logρ, Ngrad, ψs, λs, families, family_opt)
    if family_opt == 1
        retψ = zeros(2)
    else
        retψ = 0.
    end
    retλ = 0.
    for i in 1:Ngrad
        y = ys[i]
        if family_opt == 1
#             println(ψ)
            retψ = retψ .+ ForwardDiff.gradient(ψψ -> obj_comp(y, ψψ, λ, logρ, ψs, λs, families, family_opt), ψ)
            retλ = retλ + ForwardDiff.derivative(λλ -> obj_comp(y, ψ, λλ, logρ, ψs, λs, families, family_opt), λ)
        else
            retψ = retψ + ForwardDiff.derivative(ψψ -> obj_comp(y, ψψ, λ, logρ, ψs, λs, families, family_opt), ψ)
            retλ = retλ + ForwardDiff.derivative(λλ -> obj_comp(y, ψ, λλ, logρ, ψs, λs, families, family_opt), λ)
        end
    end
    
    if family_opt == 1 
        return (retψ./Ngrad), (retλ/Ngrad)
    else
        return (retψ/Ngrad), (retλ/Ngrad)
    end
end

grad_comp (generic function with 1 method)

In [176]:
function optimize_component(ψs, λs, λ_k, ψ_k, logρ, γ, Ngrad, num_iter, families, family_opt, κ)
    # start optimization
    λks = zeros(2*num_iter)
    if family_opt == 1
        ψks = zeros(2, num_iter)
    else
        ψks = zeros(num_iter)
    end

    ψ = ψ_k
    λ = λ_k

    for k=1:(2*num_iter)
        if k <= num_iter
            if family_opt == 1
                ψks[:,k] = ψ
            else
                ψks[k] = ψ
            end
        end

        λks[k] = λ
        
        # sample from κ
        ys = κ(Ngrad)
        
        # compute the gradient        
        ∇ψ, ∇λ = grad_comp(ys, ψ, λ, logρ, Ngrad, ψs, λs, families, family_opt)

        # take a step
        if k <= num_iter            
            ψ = @. ψ - γ(k) * ∇ψ
            if family_opt == 1 && ψ[2] <= 0
                ψ[2] = 0.1
            elseif family_opt == 2 && ψ <= 0
                ψ = 0.1
            end
        end

        λ = λ - (γ(k)/200) * ∇λ
        # project λ to feasible region
        if λ < 0.
            λ = 0.
        elseif λ > 1.
            λ = 1.
        end
    end

#     println("-----")
#     ys = κ(Ngrad)
#     ∇ψ, ∇λ = grad_comp(ys, ψ, λ, logρ, Ngrad, ψs, λs, families, family_opt)
#     println("ψ gradient: ", ∇ψ)
#     println("λ gradient: ", ∇λ)
#     println("-----")
    
    if family_opt == 1
        return λks[2*num_iter], ψks[:,num_iter]
    else
        return λks[2*num_iter], ψks[num_iter]
    end
end

optimize_component (generic function with 1 method)

In [139]:
function sample_from_current_mixture(Nobj, ψ, λ, family, ψs, λs, families)
    xs = zeros(Nobj)
    num_comp = size(families)[1] + 1
    
    λs = (1. - λ) .* λs
    push!(λs, λ)
    
    for i in 1:Nobj
        # identify component
        u = rand(Uniform(0,1))
        comp = 0
        for j in 1:num_comp
            if u <= sum(λs[1:j])
                comp = j
                break
            end
        end
        # sample from component
        if comp < num_comp
            if families[comp] == 1
                xs[i] = sample_gaussian_q(1, ψs[comp])[1]
            else
                xs[i] = sample_exponential_q(1, ψs[comp])[1]
            end
        else
            if family == 1
                xs[i] = sample_gaussian_q(1, ψ)[1]
            else
                xs[i] = sample_exponential_q(1, ψ)[1]
            end
        end
    end
    return xs
end

function log_density_from_current_mixture(x, ψ, λ, family, ψs, λs, families)
    ret = 0.
    for i in 1:size(families)[1]
        if families[i] == 1
            ret = ret + (1. - λ) * λs[i] * exp(logq_gaussian(x, ψs[i]))
        else
            ret = ret + (1. - λ) * λs[i] * exp(logq_exponential(x, ψs[i]))
        end
    end
    if family == 1
        ret = ret + λ * exp(logq_gaussian(x, ψ))
    else
        ret = ret + λ * exp(logq_exponential(x, ψ))
    end
    
    return ret
end

log_density_from_current_mixture (generic function with 1 method)

In [140]:
function est_obj(Nobj, ψ, λ, family, logπ, ψs, λs, families, single)
    if single
        if family == 1
            xobj = sample_gaussian_q(Nobj, ψ)
        else
            xobj = sample_exponential_q(Nobj, ψ)
        end

        ret = 0.
        for i=1:Nobj
            if family == 1
                ret += logq_gaussian(xobj[i], ψ) - logπ(xobj[i])
            else
                ret += logq_exponential(xobj[i], ψ) - logπ(xobj[i])
            end
        end
        ret /= Nobj

        return ret
    else
        xobj = sample_from_current_mixture(Nobj, ψ, λ, family, ψs, λs, families)
        ret = 0.
        for i=1:Nobj
            ret += log_density_from_current_mixture(xobj[i], ψ, λ, family, ψs, λs, families) - logπ(xobj[i])
        end
        ret /= Nobj
        
        return ret
    end
end

est_obj (generic function with 1 method)

In [224]:
function sample_from_current_mixture(Nobj, ψs, λs, families)
    xs = zeros(Nobj)
    num_comp = size(families)[1]

    for i in 1:Nobj
        # identify component
        u = rand(Uniform(0,1))
        comp = 0
        for j in 1:num_comp
            if u <= sum(λs[1:j])
                comp = j
                break
            end
        end
        # sample from component
        if families[comp] == 1
            xs[i] = sample_gaussian_q(1, ψs[comp])[1]
        else
            xs[i] = sample_exponential_q(1, ψs[comp])[1]
        end
    end
    return xs
end

sample_from_current_mixture (generic function with 2 methods)

In [363]:
function VB(logπ, γ, Ncomp, Ngrad, Nobj, num_iter, 
            grad_gaussian, grad_exponential, κ, constrain)
    transform_logπ = (x, logJ) -> logπ(x) + logJ
    logρ = x -> transform_logπ(constrain(x)...)
    
    ψs = Any[]
    families = zeros(Int8, Ncomp)
    λs = [1.]

    # fitting first component
    println("optimizing component 1 / ", Ncomp)
    ψs_gaussian = ADVI(logπ, grad_gaussian, γ, Ngrad, vec([0. 1.]), num_iter, constrain, κ, 1)
    ψs_exponential = ADVI(logπ, grad_exponential, γ, Ngrad, 0.5, num_iter, constrain, κ, 2)

    # compare objectives
    obj_gaussian = est_obj(Nobj, ψs_gaussian[:,num_iter], 0., 1, logπ, ψs, λs, families, true)
    obj_exponential = est_obj(Nobj, ψs_exponential[num_iter], 0., 2, logπ, ψs, λs, families, true)
    if obj_gaussian <= obj_exponential
        push!(ψs, ψs_gaussian[:,num_iter])
        families[1] = 1
    else
        push!(ψs, ψs_exponential[num_iter])
        families[1] = 2
    end

    println("component 1")
    println(families)
    println(ψs)
    
    # fitting remaining components
    for k in 2:Ncomp
        println("optimizing component ", k, " / ", Ncomp)
        # init new component
        λ_k = 1. / k
        # optimize new component
        λ_k_gau, ψ_k_gau = optimize_component(ψs, λs, λ_k, vec([0. 0.5]), logρ, γ, Ngrad, num_iter, families[1:k-1], 1, κ)
        λ_k_exp, ψ_k_exp = optimize_component(ψs, λs, λ_k, 7., logρ, γ, Ngrad, num_iter, families[1:k-1], 2, κ)
        # compare objectives
        obj_gaussian = est_obj(Nobj, ψ_k_gau, λ_k_gau, 1, logπ, ψs, λs, families[1:k-1], false)
        obj_exponential = est_obj(Nobj, ψ_k_exp, λ_k_exp, 2, logπ, ψs, λs, families[1:k-1], false)
        println("gau: ", obj_gaussian)
        println("exp: ", obj_exponential)
        # update parameters
        if obj_gaussian <= obj_exponential
            λs = (1. - λ_k_gau) .* λs
            push!(λs, λ_k_gau)
            push!(ψs, ψ_k_gau)
            families[k] = 1
        else
            λs = (1. - λ_k_exp) .* λs
            push!(λs, λ_k_exp)
            push!(ψs, ψ_k_exp)
            families[k] = 2
        end

        println("component ", k)
        println("param: ", ψs[k])
        println("weight: ", λs[k])
        println("-----")
    end

    return ψs, λs, families
end

VB (generic function with 1 method)

In [364]:
logπ = x -> log(0.5 * pdf(Beta(1,6), x) + 0.5 * pdf(Beta(30,10), x))
γ = k -> 0.02 /sqrt(k)
Ngrad = 100
Nobj = 100
num_iter = 3000
Ncomp = 2

Random.seed!(1)
ψs, λs, families = VB(logπ, γ, Ncomp, Ngrad, Nobj, num_iter, 
                        grad_gaussian, grad_exponential, κ, constrain);

optimizing component 1 / 2
component 1
Int8[2, 0]
Any[0.23418498018439665]
optimizing component 2 / 2
gau: 1.2609888931655608
exp: 4.645309650350763
component 2
param: [-1.115552618763077, 0.2636297188544139]
weight: 0.28588861740644755
-----


In [367]:
samples = sample_from_current_mixture(5000, ψs, λs, families)
histogram(samples, normed=true,bins=50, alpha=0.3, label = "approximation")
plot!(0:0.01:1, x -> exp(logπ(x)), label = "target", linewidth=1.5)
png("both_multiple")

In [299]:
function VB_gaussian_only(logπ, γ, Ncomp, Ngrad, Nobj, num_iter, 
                          grad_gaussian, grad_exponential, κ, constrain)
    transform_logπ = (x, logJ) -> logπ(x) + logJ
    logρ = x -> transform_logπ(constrain(x)...)
    
    ψs = Any[]
    families = zeros(Int8, Ncomp)
    λs = [1.]

    # fitting first component
    println("optimizing component 1 / ", Ncomp)
    ψs_gaussian = ADVI(logπ, grad_gaussian, γ, Ngrad, vec([0. 1.]), num_iter, constrain, κ, 1)
    ψs_exponential = ADVI(logπ, grad_exponential, γ, Ngrad, 0.5, num_iter, constrain, κ, 2)

    # compare objectives
    obj_gaussian = est_obj(Nobj, ψs_gaussian[:,num_iter], 0., 1, logπ, ψs, λs, families, true)
    push!(ψs, ψs_gaussian[:,num_iter])
    families[1] = 1

    println("component 1")
    println(families)
    println(ψs)
    
    # fitting remaining components
    for k in 2:Ncomp
        println("optimizing component ", k, " / ", Ncomp)
        # init new component
        λ_k = 1. / k
        # optimize new component
        λ_k_gau, ψ_k_gau = optimize_component(ψs, λs, λ_k, vec([0. 0.5]), logρ, γ, Ngrad, num_iter, families[1:k-1], 1, κ)
        λs = (1. - λ_k_gau) .* λs
        push!(λs, λ_k_gau)
        push!(ψs, ψ_k_gau)
        families[k] = 1

        println("component ", k)
        println("param: ", ψs[k])
        println("weight: ", λs[k])
        println("-----")
    end

    return ψs, λs, families
end

VB_gaussian_only (generic function with 1 method)

In [302]:
logπ = x -> log(0.5 * pdf(Beta(1,6), x) + 0.5 * pdf(Beta(30,10), x))
# logπ = x -> logpdf(Beta(50.,2.), x)
γ = k -> 0.1 /sqrt(k)
Ngrad = 100
Nobj = 100
num_iter = 3000
Ncomp = 2

Random.seed!(1)
ψs, λs, families = VB_gaussian_only(logπ, γ, Ncomp, Ngrad, Nobj, num_iter, 
            grad_gaussian, grad_exponential, κ, constrain);

optimizing component 1 / 2
component 1
Int8[1, 0]
Any[[1.4657170491878704, 1.667031311389476]]
optimizing component 2 / 2
component 2
param: [-1.149992627941423, 0.33093973130258136]
weight: 0.4854751263221256
-----


In [304]:
samples = sample_from_current_mixture(5000, ψs, λs, families)
histogram(samples, normed=true,bins=50, alpha=0.3, label = "approximation")
plot!(0:0.01:1, x -> exp(logπ(x)), label = "target", linewidth=1.5)
png("gauss_multiple")